In [5]:
from pdf2image import convert_from_path

In [6]:
pages = convert_from_path(r'docs\patient_details\pd_1.pdf', poppler_path=r'C:\poppler-23.08.0\Library\bin')

In [7]:
pages

[<PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1867x2000 at 0x1800C1E0100>,
 <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=2000x1398 at 0x1800C1E0130>]

In [8]:
pages[0].show()

In [9]:
pages[1].show()

In [10]:
len(pages)

2

In [12]:
import pytesseract

In [13]:
pytesseract.pytesseract.tesseract_cmd=r'C:\Program Files\Tesseract-OCR\tesseract.exe'
text1=pytesseract.image_to_string(pages[0],lang='eng')
text2=pytesseract.image_to_string(pages[1],lang='eng')
print(text1)
print(text2)

47/12/2020

Patient Medical Record

Patient Information Birth Date
Kathy Crawford May 6 1972
(737) 988-0851 Weight
9264 Ash Dr 95
New York City, 10005 .
United States Height:
190
In Case of Emergency
m _ a _
Simeone Crawford 9266 Ash Dr
New York City, New York, 10005
Home phone United States
(990) 375-4621
Work phone
Genera! Medical History
. : a ee

Chicken Pox (Varicella):

IMMUNE

Have you had the Hepatitis B vaccination?

No

List any Medical Problems (asthma, seizures, headaches):

Migraine
a

’ Name of Insurance Company:

Random tnsuarance Company : 4789 Bollinger Rd
Policy Number: —————EE7

7115207313 Expiry Date:
30 December 2020
Do you have medical insurance?

Yes

Medical Insurance Details

List any allergies:

Peanuts

List any medication taken regularly:

Triptans


In [59]:
import cv2
import numpy as np
def preprocess_image(img):
    gray = cv2.cvtColor(np.array(img), cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, None, fx=2, fy=2, interpolation=cv2.INTER_LINEAR) # noqa
    processed_img = cv2.adaptiveThreshold(resized, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 61, 11) # noqa
    return processed_img

In [60]:
img = preprocess_image(pages[0])

from PIL import Image
Image.fromarray(img).show()


text = pytesseract.image_to_string(img, lang='eng')
print(text)


17/12/2020

Patient Medical Record . : :

Patient Information

Birth Date
Kathy Crawford May 6 1972
(737) 988-0851 Weight:
9264 Ash Dr 95
New York City, 10005 a
United States Height:
190
In Case of Emergency
en oe
Simeone Crawford 9266 Ash Dr
New York City, New York, 10005
Home phone United States
(990) 375-4621
Work phone
Genera! Medical History
I a
Chicken Pox (Varicella): Measies:
IMMUNE IMMUNE

Have you had the Hepatitis B vaccination?

No

List any Medical Problems (asthma, seizures, headaches):

Migraine


<h2>Extract Name</h2>

In [61]:
import re

pattern = 'Patient Information(.*?)\(\d{3}\)'

matches = re.findall(pattern, text, flags=re.DOTALL)
matches

['\n\nBirth Date\nKathy Crawford May 6 1972\n']

In [48]:
matches[0].strip()

'Birth Date\n\nKathy Crawford May 6 1972'

In [49]:
match = matches[0].replace("Birth Date","").strip()
match

'Kathy Crawford May 6 1972'

In [50]:
pattern = '((Jan|Feb|March|April|May|June|July|Aug|Sep|Oct|Nov|Dec)[ \d]+)'

date_matches = re.findall(pattern, match)
date = date_matches[0][0]
date

'May 6 1972'

In [51]:
match.replace(date, '').strip()

'Kathy Crawford'

In [52]:
def remove_noise_from_name(name):
    name = name.replace('Birth Date','').strip()
    date_pattern = '((Jan|Feb|March|April|May|June|July|Aug|Sep|Oct|Nov|Dec)[ \d]+)'
    date_matches = re.findall(date_pattern, name)
    
    if date_matches:
        date = date_matches[0][0]
        name = name.replace(date, '').strip()
    
    return name

In [53]:
name = '\n\n \n\n \n\nBirth Date\nKathy Crawford May 6 1972\n'

name = remove_noise_from_name(name)
name

'Kathy Crawford'

<h2>Extract phone</h2>

In [54]:
pattern = 'Patient Information(.*?)(\(\d{3}\) \d{3}-\d{4})'

matches = re.findall(pattern, text, flags=re.DOTALL)
matches

[(' Birth Date\n\nKathy Crawford May 6 1972\n\n', '(737) 988-0851')]

In [55]:
matches[0][1]

'(737) 988-0851'

<h2>Extract vaccine</h2>

In [62]:
pattern = 'Have you had the Hepatitis B vaccination\?.*(Yes|No)'

matches = re.findall(pattern, text, flags=re.DOTALL)
matches

['No']

<h2>Extract Medical Problems</h2>

In [63]:
pattern = 'List any Medical Problems \(asthma, seizures, headaches\):(.*)'

matches = re.findall(pattern, text, flags=re.DOTALL)
matches[0].strip()

'Migraine'

In [64]:
import re

pattern = r'List any Medical Problems \(asthma, seizures, headaches\):(.*)'
matches = re.findall(pattern, text, flags=re.DOTALL)

if matches:
    first_match = matches[0].strip()
    # Do something with first_match
else:
    print("No matches found.")

first_match

'Migraine'